In [ ]:
!pip install dabl

In [56]:
import os
import json
import ast
import dabl
import math
import pandas as pd

In [57]:
os.getcwd()


'e:\\Workspaces\\SASRec.pytorch\\notebooks'

In [58]:
ROOT_DIR = os.path.join(os.getcwd(), "..", "data", "steam")

In [59]:
ROOT_DIR

'e:\\Workspaces\\SASRec.pytorch\\notebooks\\..\\data\\steam'

In [60]:
os.path.exists(os.path.join(ROOT_DIR, "steam_games.json"))

True

In [61]:
with open(os.path.join(ROOT_DIR, "steam_games.json")) as json_file:
    lines = json_file.read().splitlines()
records = [ast.literal_eval(line) for line in lines]

In [62]:
raw_df = pd.DataFrame(records)

In [63]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32135 entries, 0 to 32134
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   publisher       24083 non-null  object 
 1   genres          28852 non-null  object 
 2   app_name        32133 non-null  object 
 3   title           30085 non-null  object 
 4   url             32135 non-null  object 
 5   release_date    30068 non-null  object 
 6   tags            31972 non-null  object 
 7   discount_price  225 non-null    float64
 8   reviews_url     32133 non-null  object 
 9   specs           31465 non-null  object 
 10  price           30758 non-null  object 
 11  early_access    32135 non-null  bool   
 12  id              32133 non-null  object 
 13  developer       28836 non-null  object 
 14  sentiment       24953 non-null  object 
 15  metascore       2677 non-null   object 
dtypes: bool(1), float64(1), object(14)
memory usage: 3.7+ MB


In [64]:
df = raw_df[['id', 'sentiment', 'genres', 'specs', 'price', 'early_access']]

In [65]:
#fix id
df.dropna(subset = ['id'], inplace=True)

<ipython-input-65-2bc2d88906ee>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset = ['id'], inplace=True)


In [66]:
#fix sentiment
sentiments = ['Unknown', 'Overwhelmingly Negative', "Very Negative", "Negative", "Mostly Negative", "Mixed", "Mostly Positive", "Positive" , "Very Positive" ,"Overwhelmingly Positive"]
sentiments_dict = {
    'Unknown': -1,
    'Overwhelmingly Negative': 0,
    'Very Negative': 1,
    'Negative': 2,
    'Mostly Negative': 3,
    'Mixed': 4,
    'Mostly Positive': 5,
    'Positive': 6,
    'Very Positive': 7,
    'Overwhelmingly Positive': 8,
}
df['sentiment'] = df['sentiment'].fillna('Unknown')
df['sentiment'] = df['sentiment'].apply(lambda x: x if x in sentiments else 'Unknown')
df['sentiment'] = df['sentiment'].apply(lambda x: sentiments_dict[x])
df['sentiment'] = df['sentiment'].astype(int)

<ipython-input-66-261ac1949ad6>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].fillna('Unknown')
<ipython-input-66-261ac1949ad6>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sentiment'] = df['sentiment'].apply(lambda x: x if x in sentiments else 'Unknown')
<ipython-input-66-261ac1949ad6>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the d

In [67]:
#fix genres
genres = set()
for item in list(raw_df["genres"].dropna()):        
    genres = genres | set(item)
print(len(genres))

def is_in_genere(genre, x):
    try:
        return 1 if genre in x else 0
    except:
        return 0

for genre in genres:
    df[f'genre_{genre}'] = df['genres'].apply(lambda x: is_in_genere(genre=genre, x=x))

df.drop(columns=['genres'],inplace=True)

22
<ipython-input-67-d9d1c6b5298e>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'genre_{genre}'] = df['genres'].apply(lambda x: is_in_genere(genre=genre, x=x))
e:\Workspaces\SASRec.pytorch\.venv_sas_rec\lib\site-packages\pandas\core\frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


In [68]:
#fix specs
specs = set()
for item in list(raw_df["specs"].dropna()):        
    specs = specs | set(item)
print(len(specs))

def is_in_spec(spec, x):
    try:
        return 1 if spec in x else 0
    except:
        return 0

for spec in specs:
    df[f'spec_{spec}'] = df['specs'].apply(lambda x: is_in_spec(spec, x))

df.drop(columns=['specs'],inplace=True)

<ipython-input-68-c31250a17a95>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'spec_{spec}'] = df['specs'].apply(lambda x: is_in_spec(spec, x))
40


In [69]:
#fix price
# strange_prices = ['Free to Play', 'nan','Free','Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Third-party', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Starting at $499.00', 'Starting at $449.00', 'Free to Try', 'Free Movie', 'Free to Use']
free_prices = ['Free to Play', 'Free', 'Free Demo', 'Play for Free!', 'Install Now', 'Play WARMACHINE: Tactics Demo', 'Free Mod', 'Install Theme', 'Play Now', 'Free HITMAN™ Holiday Pack', 'Play the Demo', 'Free to Try', 'Free Movie', 'Free to Use', 'Free To Play	', "'Free To Play'"]
free_prices = free_prices + [df.iloc[1]['price']]
df['price'] = df['price'].apply(lambda x: 0 if x in free_prices else x)
df['price'] = df['price'].apply(lambda x: 0 if x == 'Third-party' else x)
df['price'] = df['price'].apply(lambda x: 499.0 if x == 'Starting at $499.00' else x)
df['price'] = df['price'].apply(lambda x: 449.0 if x == 'Starting at $449.00' else x)
df.dropna(subset = ['price'], inplace=True)

df['price']=df['price'].apply(lambda x:round(x,2))

<ipython-input-69-81a964d19bf9>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].apply(lambda x: 0 if x in free_prices else x)
<ipython-input-69-81a964d19bf9>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['price'] = df['price'].apply(lambda x: 0 if x == 'Third-party' else x)
<ipython-input-69-81a964d19bf9>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [70]:
# fix early_access
df['early_access'] = df['early_access'].apply(lambda x: 1 if x else 0)

<ipython-input-70-08b71fb7b30e>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['early_access'] = df['early_access'].apply(lambda x: 1 if x else 0)


In [71]:
df[df.isnull().any(axis=1)]

,id,sentiment,price,early_access,genre_Free to Play,genre_Action,genre_Racing,genre_Accounting,genre_Photo Editing,genre_Video Production,...,spec_Windows Mixed Reality,spec_Partial Controller Support,spec_Gamepad,spec_Seated,spec_Local Co-op,spec_Full controller support,spec_SteamVR Collectibles,spec_Captions available,spec_Co-op,spec_In-App Purchases


In [72]:
df

,id,sentiment,price,early_access,genre_Free to Play,genre_Action,genre_Racing,genre_Accounting,genre_Photo Editing,genre_Video Production,...,spec_Windows Mixed Reality,spec_Partial Controller Support,spec_Gamepad,spec_Seated,spec_Local Co-op,spec_Full controller support,spec_SteamVR Collectibles,spec_Captions available,spec_Co-op,spec_In-App Purchases
0,761140,-1,4.99,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,643980,5,0.00,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,670290,5,0.00,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,767400,-1,0.99,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,773570,-1,2.99,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32130,773640,-1,1.99,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32131,733530,-1,4.99,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32132,610660,-1,1.99,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
32133,658870,-1,4.99,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [73]:
df.to_csv(os.path.join(ROOT_DIR, "items_info.csv"), index=False)